In [1]:
import pandas as pd
import re
import json
import csv
from datetime import datetime
import logging

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
orders_json = [
    {
        "Order": "119B71CC",
        "Delivered": "The order was delivered at 10:16 PM on October 4, 2023.",
        "Pick Up Location": "5004 Wesley St, Greenville, TX 75402, USA",
        "Order Details": {
            "2 \u00d7 Bud Ice Lager Beer Can (25 oz) (Domestic Beer NT)": "$8.78",
            " 3 \u00d7 Steel Reserve Original Can (25 oz) (Flavored Malt Beverages NT)": "$13.17"
        },
        "Subtotal": "$21.95",
        "Subtotal Tax": "$1.81",
        "Commission (16%)": "-$3.51",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$20.25",
        "Transaction #8134054141 - Delivery": "$20.25"
    },
    {
        "Order": "CC36E208",
        "Delivered": "The order was delivered at 9:18 PM on October 4, 2023.",
        "Pick Up Location": "",
        "Order Details": {
            "1 \u00d7 Miller Lite Beer Can (12 oz x 18 ct) (Domestic Beer NT)": "$28.79"
        },
        "Subtotal": "$28.79",
        "Subtotal Tax": "$2.38",
        "Commission (16%)": "-$4.61",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$0.00"
    },
    {
        "Order": "C7B1DB1A Could Not Deliver The Dasher attempted to deliver to your customer, but couldn't complete the delivery.",
        "Pick Up Location": "5004 Wesley St, Greenville, TX 75402, USA",
        "Order Details": {
            "1 \u00d7 Miller Lite Beer Can (12 oz x 18 ct) (Domestic Beer NT)": "$28.79"
        },
        "Subtotal": "$28.79",
        "Subtotal Tax": "$2.38",
        "Commission (23%)": "-$6.62",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$0.00",
        "Transaction #8133401784 - Delivery": "$24.55",
        "Transaction #8133791059 - Delivery": "-$24.55"
    },
    {
        "Order": "455421D6",
        "Delivered": "Error Charge The customer reported one or more issues resulting in an error charge of $11.52. If you believe this was an error, you can dispute the error charge by October 18, 2023.",
        "Pick Up Location": "1402 Williams Dr, Georgetown, TX 78628, USA",
        "Order Details": {
            "1 \u00d7 Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: TXB Root Beer": "$0.00",
            " 1 \u00d7 Chicken Tender Plate (3 pcs) (Chicken TXB#65) Dipping Sauce: Chicken Dipping Sauce Dipping Sauce: Ranch Dipping Sauce Dipping Sauce: Boom-Boom Dipping Sauce Choice of Sides: Mac & Cheese Choice of Sides: Mashed Potatoes & Gravy Original or Spicy: Spicy": "$12.49"
        },
        "Subtotal": "-$10.49 Error Charge Tax -$1.03 Error Charge Total -$11.52 Order Total $11.52",
        "Commission (16%)": "-$2.00 Order Total $11.52 Error Charges 1 \u00d7 Chicken Tender Plate (3 pcs) (Chicken TXB#65) Dipping Sauce: Chicken Dipping Sauce Dipping Sauce: Ranch Dipping Sauce Dipping Sauce: Boom-Boom Dipping Sauce Choice of Sides: Mac & Cheese Choice of Sides: Mashed Potatoes & Gravy Original or Spicy: Spicy Food Prepared Unsafely -$10.49 Error Charge",
        "Estimated Payout": "$0.00",
        "Transaction #8132060968 - Delivery": "$11.52 Transaction #8132372658 - Error Charges -$11.52"
    },
    {
        "Order": "11518E3F",
        "Delivered": "The order was delivered at 6:01 PM on October 4, 2023.",
        "Pick Up Location": "1001 S State Hwy 16, Fredericksburg, TX 78624, USA",
        "Order Details": {
            "1 \u00d7 Ice Large (Fountain, Frozen, & Tea)": "$7.49"
        },
        "Subtotal": "$7.49",
        "Subtotal Tax": "$0.62",
        "Commission (16%)": "-$1.20",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$6.91",
        "Transaction #8131754028 - Delivery": "$6.91"
    },
    {
        "Order": "717EFC23",
        "Delivered": "The order was delivered at 1:32 PM on October 4, 2023.",
        "Pick Up Location": "1001 W Van Alstyne Pkwy, Van Alstyne, TX 75495, USA",
        "Order Details": {
            "2 \u00d7 TXB Strawberry White Tea Bottle (16oz) (Tea TX)": "$5.18",
            " 1 \u00d7 Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: Sprite": "$0.00",
            " 1 \u00d7 Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: Mountain Dew": "$0.00",
            " 1 \u00d7 Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: Gatorade Lemon-Lime": "$0.00",
            " 1 \u00d7 Gatorade Lime Cucumber Bottle (28 oz) (Sports Drinks TX)": "$4.79",
            " 1 \u00d7 Arizona Green Tea Bottle (34oz) (Tea TX)": "$4.39"
        },
        "Subtotal": "$14.36",
        "Subtotal Tax": "$0.82",
        "Commission (23%)": "-$3.30",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$11.88",
        "Transaction #8129725522 - Delivery": "$11.88"
    },
    {
        "Order": "1187C4F6",
        "Delivered": "The order was delivered at 1:11 PM on October 4, 2023.",
        "Pick Up Location": "1001 S State Hwy 16, Fredericksburg, TX 78624, USA",
        "Order Details": {
            "1 \u00d7 Monster Energy Drink Can (16 oz) (Energy Drinks TX)": "$4.79",
            " 1 \u00d7 One Bars Chocolate Chip Cookie Dough (2.12oz) (Nutritional Bars TX)": "$4.99",
            " 1 \u00d7 Jack Link's Cheese & Beef Steak (1.2oz) (Meat Snacks TX)": "$3.99",
            " 1 \u00d7 C4 Starburst Strawberry Can (16oz) (Energy Drinks TX)": "$4.79"
        },
        "Subtotal": "$18.56",
        "Subtotal Tax": "$1.20",
        "Commission (23%)": "-$4.27",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$15.49",
        "Transaction #8129629020 - Delivery": "$15.49"
    },
    {
        "Order": "56F1F9DF",
        "Delivered": "The order was delivered at 12:35 PM on October 4, 2023.",
        "Pick Up Location": "145 Lehman Rd, Kyle, TX 78640, USA",
        "Order Details": {
            "2 \u00d7 Peanut Butter Chunk Cookie (Pastries & Desserts)": "$4.98",
            " 1 \u00d7 Chicken Tender Plate (3 pcs) (Chicken TXB#68) Dipping Sauce: Ranch Dipping Sauce Dipping Sauce: Honey Mustard Dipping Sauce Dipping Sauce: Boom-Boom Dipping Sauce Choice of Sides: Mac & Cheese Choice of Sides: Mashed Potatoes & Gravy Original or Spicy: Spicy": "$12.49"
        },
        "Subtotal": "$17.47",
        "Subtotal Tax": "$1.44",
        "Commission (16%)": "-$2.80",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$16.11",
        "Transaction #8129341631 - Delivery": "$16.11"
    },
    {
        "Order": "09CCEF99",
        "Delivered": "The order was delivered at 11:31 AM on October 4, 2023.",
        "Pick Up Location": "201 S 9th St, Durant, OK 74701, USA",
        "Order Details": {
            "3 \u00d7 Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: Mountain Dew": "$0.00",
            " 1 \u00d7 Dr Pepper Cherry Can (12 pk-12 oz) (Soda OK)": "$11.29"
        },
        "Subtotal": "$11.29",
        "Subtotal Tax": "$1.06",
        "Commission (16%)": "-$1.81",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$9.48",
        "Transaction #8128900737 - Delivery": "$9.48"
    },
    {
        "Order": "806C7274",
        "Delivered": "The order was delivered at 10:54 AM on October 4, 2023.",
        "Pick Up Location": "2503 TX-349, Midland, TX 79706, USA",
        "Order Details": {
            "1 \u00d7 Cheez-It Original (3 oz) (Crackers & Pretzels TX)": "$3.39",
            " 1 \u00d7 Hostess Chocolate Donettes Frosted (3oz) (Donuts TX)": "$3.79",
            " 2 \u00d7 Zapps Chips Voodoo Heat (2.5oz) (Chips & Popcorn TX)": "$6.98",
            " 1 \u00d7 Snickers King Size (3.29oz) (Chocolate TX)": "$3.79",
            " 4 \u00d7 Red Bull Red Edition Watermelon Can (12 oz) (Energy Drinks TX)": "$23.16"
        },
        "Subtotal": "$41.11",
        "Subtotal Tax": "$2.73",
        "Commission (16%)": "-$6.58",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$37.26",
        "Transaction #8128606134 - Delivery": "$37.26"
    },
    {
        "Order": "4D6A8B19",
        "Delivered": "The order was delivered at 10:38 AM on October 4, 2023.",
        "Pick Up Location": "440 W Bandera Rd, Boerne, TX 78006, USA",
        "Order Details": {
            "1 \u00d7 TXB Green Tea Bottle (16oz) (Tea TX)": "$2.59",
            " 1 \u00d7 TXB Fredericksburg Peach Sweet Tea Bottle (16oz) (Tea TX)": "$3.29"
        },
        "Subtotal": "$5.88",
        "Subtotal Tax": "$0.21",
        "Commission (20%)": "-$1.18",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$4.91",
        "Transaction #8128576360 - Delivery": "$4.91"
    },
    {
        "Order": "4814C0EE",
        "Delivered": "The order was delivered at 8:07 AM on October 4, 2023.",
        "Pick Up Location": "1402 Williams Dr, Georgetown, TX 78628, USA",
        "Order Details": {
            "3 \u00d7 Big TX Burrito (Breakfast Tacos & Quesadillas) Salsa: Salsa Verde": "$8.67"
        },
        "Subtotal": "$8.67",
        "Subtotal Tax": "$0.72",
        "Commission (16%)": "-$1.39",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$8.00",
        "Transaction #8128036707 - Delivery": "$8.00"
    },
    {
        "Order": "7967144C",
        "Delivered": "The order was delivered at 7:53 AM on October 4, 2023.",
        "Pick Up Location": "5004 Wesley St, Greenville, TX 75402, USA",
        "Order Details": {
            "1 \u00d7 Monster Ultra Peachy Keen Can (16oz) (Energy Drinks TX)": "$4.79",
            " 1 \u00d7 Tropicana Apple Juice (12oz) (Juice TX)": "$3.89"
        },
        "Subtotal": "$8.68",
        "Subtotal Tax": "$0.40",
        "Commission (16%)": "-$1.39",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$7.69",
        "Transaction #8128003710 - Delivery": "$7.69"
    },
    {
        "Order": "EB4E3B27",
        "Delivered": "The order was delivered at 7:26 AM on October 4, 2023.",
        "Pick Up Location": "201 S 9th St, Durant, OK 74701, USA",
        "Order Details": {
            "1 \u00d7 Sprite Bottle (20 oz) (Soda OK)": "$3.19",
            " 1 \u00d7 Bacon Egg Cheese Biscuit (Breakfast Sandwiches & Bowls)": "$4.89",
            " 2 \u00d7 Monster Java Mean Bean Can (15oz) (Coffee OK)": "$11.38"
        },
        "Subtotal": "$19.46",
        "Subtotal Tax": "$1.82",
        "Commission (23%)": "-$4.48",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$14.98",
        "Transaction #8127972503 - Delivery": "$14.98"
    },
    {
        "Order": "D8CEA439",
        "Delivered": "The order was delivered at 12:08 AM on October 4, 2023.",
        "Pick Up Location": "2320 W Main St, Durant, OK 74701, USA",
        "Order Details": {
            "1 \u00d7 Pepsi Can (12pk 12oz) (Soda OK)": "$11.29"
        },
        "Subtotal": "$11.29",
        "Subtotal Tax": "$1.06",
        "Commission (16%)": "-$1.81",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$9.48",
        "Transaction #8127379901 - Delivery": "$9.48"
    },
    {
        "Order": "AD3E1757",
        "Delivered": "The order was delivered at 12:07 AM on October 4, 2023.",
        "Pick Up Location": "5004 Wesley St, Greenville, TX 75402, USA",
        "Order Details": {
            "1 \u00d7 Bai Boost Strawberry Watermelon Bottle (18oz) (Water TX)": "$4.19",
            " 2 \u00d7 C4 Starburst Strawberry Can (16oz) (Energy Drinks TX)": "$9.58"
        },
        "Subtotal": "$13.77",
        "Subtotal Tax": "$1.14",
        "Commission (23%)": "-$3.17",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$11.74",
        "Transaction #8127420369 - Delivery": "$11.74"
    }
]

In [3]:
master_dataset_file_path = '/Users/ekim/workspace/personal/dd-bot/dev/store_list.xlsx'


In [4]:
def read_masterdataset_excel():
    master_df = pd.read_excel(master_dataset_file_path, index_col=0)
    return master_df

master_df = read_masterdataset_excel()


def add_cols_to_masterset_df(master_df):

    # Cast int64 default dtype cols to strings for proper JOINs
    master_df['Site #'] = master_df['Site #'].astype(str)
    master_df['Zip'] = master_df['Zip'].astype(str)
    return master_df

master_df = add_cols_to_masterset_df(master_df)
master_df

,Site #,Site Description,Address,City,County,State,Zip,Notes
Store #,,,,,,,,
1,1,Kwik Chek #1,2226 North Center,Bonham,Fannin,TX,75418,NaN
3,3,Kwik Chek #3,971 FM 273,Bonham,Fannin,TX,75418,NaN
5,5,Kwik Chek #5,NaN,Bonham,Fannin,TX,75418,Not Open Yet
6,6,Kwik Chek #6,816 North Center,Bonham,Fannin,TX,75418,NaN
7,7,Kwik Chek #7,101 Hwy 69,Celeste,Hunt,TX,75423,NaN
8,8,Kwik Chek #8,308 Hwy 69,Leonard,Fannin,TX,75452,NaN
10,10,Kwik Chek #10,1741 North Hwy 121,Bonham,Fannin,TX,75418,NaN
12,12,Kwik Chek #12,102 S Hwy 78,Achille,Bryan,OK,74720,NaN
13,13,Kwik Chek #13,102 West Main,Calera,Bryan,OK,74730,NaN


In [13]:
def remove_incomplete_orders(orders):
    complete_orders = []
    for order in orders:
        if len(order) >= 7:
            complete_orders.append(order)
    return complete_orders

_orders_json = remove_incomplete_orders(orders_json)
_orders_json

[{'Order': '119B71CC',
  'Delivered': 'The order was delivered at 10:16 PM on October 4, 2023.',
  'Pick Up Location': '5004 Wesley St, Greenville, TX 75402, USA',
  'Order Details': {'2 × Bud Ice Lager Beer Can (25 oz) (Domestic Beer NT)': '$8.78',
   ' 3 × Steel Reserve Original Can (25 oz) (Flavored Malt Beverages NT)': '$13.17'},
  'Subtotal': '$21.95',
  'Subtotal Tax': '$1.81',
  'Commission (16%)': '-$3.51',
  'Total Customer Refund': '-$0.00',
  'Estimated Payout': '$20.25',
  'Transaction #8134054141 - Delivery': '$20.25'},
 {'Order': 'CC36E208',
  'Delivered': 'The order was delivered at 9:18 PM on October 4, 2023.',
  'Pick Up Location': '000 Placeholder Address',
  'Order Details': {'1 × Miller Lite Beer Can (12 oz x 18 ct) (Domestic Beer NT)': '$28.79'},
  'Subtotal': '$28.79',
  'Subtotal Tax': '$2.38',
  'Commission (16%)': '-$4.61',
  'Total Customer Refund': '-$0.00',
  'Estimated Payout': '$0.00'},
 {'Order': "C7B1DB1A Could Not Deliver The Dasher attempted to deliver

In [16]:
import logging

def validate_and_clean_order_id(order):
    if 'Order' not in order:
        logging.error(f' Missing "Order" key in the order dictionary: {order}')
        return None

    order_id = order['Order']
    if len(order_id) > 8:
        logging.warning(f' OrderID is too long. Truncating OrderID to 8 characters...')
        return order_id[:8]
    elif len(order_id) == 0:
        logging.error(f' OrderID is missing. Please evaluate the order: \n{order}')
        return None
    else:
        return order_id

def validate_and_clean_store_addrs(order):
    if 'Pick Up Location' not in order:
        logging.error(f' Missing "Pick Up Location" key in the order dictionary: {order}')
        return None

    store_addrs = order['Pick Up Location']
    if len(store_addrs) == 0:
        logging.warning(f' Pick Up location is missing from Order. Adding placeholder...')
        store_addrs = '000 Placeholder Address'
    return store_addrs

def order_id_to_pickup_location_fixed(orders_json):
    order_id_to_pickup_location_map = {}
    if not orders_json:
        logging.error(' orders_json is empty or None. Please provide a valid orders_json.')
        return None

    for order in orders_json:
        order_id = validate_and_clean_order_id(order)
        if order_id is None:
            continue

        store_addrs = validate_and_clean_store_addrs(order)
        if store_addrs is None:
            continue

        order_id_to_pickup_location_map[order_id] = store_addrs

    if not order_id_to_pickup_location_map:
        logging.error(' No valid orders found. Returning None.')
        return None

    return order_id_to_pickup_location_map

# Usage example
# _orders_json should be your actual JSON object
result = order_id_to_pickup_location_fixed(_orders_json)
result

{'119B71CC': '5004 Wesley St, Greenville, TX 75402, USA',
 'CC36E208': '000 Placeholder Address',
 'C7B1DB1A': '5004 Wesley St, Greenville, TX 75402, USA',
 '455421D6': '1402 Williams Dr, Georgetown, TX 78628, USA',
 '11518E3F': '1001 S State Hwy 16, Fredericksburg, TX 78624, USA',
 '717EFC23': '1001 W Van Alstyne Pkwy, Van Alstyne, TX 75495, USA',
 '1187C4F6': '1001 S State Hwy 16, Fredericksburg, TX 78624, USA',
 '56F1F9DF': '145 Lehman Rd, Kyle, TX 78640, USA',
 '09CCEF99': '201 S 9th St, Durant, OK 74701, USA',
 '806C7274': '2503 TX-349, Midland, TX 79706, USA',
 '4D6A8B19': '440 W Bandera Rd, Boerne, TX 78006, USA',
 '4814C0EE': '1402 Williams Dr, Georgetown, TX 78628, USA',
 '7967144C': '5004 Wesley St, Greenville, TX 75402, USA',
 'EB4E3B27': '201 S 9th St, Durant, OK 74701, USA',
 'D8CEA439': '2320 W Main St, Durant, OK 74701, USA',
 'AD3E1757': '5004 Wesley St, Greenville, TX 75402, USA'}

In [12]:
def order_id_to_pickup_location_fixed(orders_json):
    order_id_to_pickup_location_map = {}
    for order in orders_json:
        order_id = order['Order']
        if len(order_id) > 8:
            logging.warning(f' OrderID: \n******\n{order_id}\n******\nis too long.\nRepairing OrderID....')
            order_id = order_id[:8]
        elif len(order_id) == 0:
            logging.error(f' OrderID is missing. Please evaluate the order: \n{order}\n')
            return
        store_addrs = order['Pick Up Location']
        if len(store_addrs) == 0:
            logging.warning(f' Pick Up location is missing from Order. Please evaluate the order.\nAdding placeholder...')
            store_addrs = '000 Placeholder Address'
            order['Pick Up Location'] = store_addrs
            logging.warning(f' Added placeholder to Pick Up Location \n******\n{order}\n******')

        order_id_to_pickup_location_map[order_id] = store_addrs
    return order_id_to_pickup_location_map
    
order_id_to_pickup_location_fixed(_orders_json)

******
C7B1DB1A Could Not Deliver The Dasher attempted to deliver to your customer, but couldn't complete the delivery.
******
is too long.
Repairing OrderID....


{'119B71CC': '5004 Wesley St, Greenville, TX 75402, USA',
 'CC36E208': '000 Placeholder Address',
 'C7B1DB1A': '5004 Wesley St, Greenville, TX 75402, USA',
 '455421D6': '1402 Williams Dr, Georgetown, TX 78628, USA',
 '11518E3F': '1001 S State Hwy 16, Fredericksburg, TX 78624, USA',
 '717EFC23': '1001 W Van Alstyne Pkwy, Van Alstyne, TX 75495, USA',
 '1187C4F6': '1001 S State Hwy 16, Fredericksburg, TX 78624, USA',
 '56F1F9DF': '145 Lehman Rd, Kyle, TX 78640, USA',
 '09CCEF99': '201 S 9th St, Durant, OK 74701, USA',
 '806C7274': '2503 TX-349, Midland, TX 79706, USA',
 '4D6A8B19': '440 W Bandera Rd, Boerne, TX 78006, USA',
 '4814C0EE': '1402 Williams Dr, Georgetown, TX 78628, USA',
 '7967144C': '5004 Wesley St, Greenville, TX 75402, USA',
 'EB4E3B27': '201 S 9th St, Durant, OK 74701, USA',
 'D8CEA439': '2320 W Main St, Durant, OK 74701, USA',
 'AD3E1757': '5004 Wesley St, Greenville, TX 75402, USA'}